In [2]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit
from pyspark.sql.window import Window
import duckdb

In [3]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/16 17:54:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [42]:
# compravenda_sup
DF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:formatted_zone/compravenda_sup.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", "SELECT * FROM RDD") \
    .load()

# verificar duplicats
print(f"Hi ha {DF.count() - DF.distinct().count()} valors duplicats al DataFrame 'formatted_zone/compravenda_sup.db' de {DF.count()} valors totals")

# substituir "--" por NA
DF = DF.withColumn('Nombre', F.when(col('Nombre') == '--', None).otherwise(col('Nombre')))
# verificar NA
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs')
# imprimir toda la informació que té un NA
valorsnull = DF.filter(DF.Nombre.isNull())

# passar a numeric
DF = DF.withColumn('Nombre', DF['Nombre'].cast('float'))

# Imputar NA de Nombre amb la mitjana de "Nombre" per "Nom_barri"
mean_nombre_nom_barri = DF.groupBy('Nom_barri').agg(F.mean('Nombre').alias('mean_nombre_nom_barri'))

DF = DF.join(mean_nombre_nom_barri, on='Nom_barri', how='left')
DF = DF.withColumn('Nombre', F.when(DF['Nombre'].isNull(), DF['mean_nombre_nom_barri']).otherwise(DF['Nombre']))
# Verificar NAs després de la imputació per barri
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs en barri')

# Imputar NA de Nombre amb la mitjana de "Nombre" per "Nom_districte"
mean_nombre_nom_districte = DF.groupBy('Nom_Districte').agg(F.mean('Nombre').alias('mean_nombre_nom_districte'))

DF = DF.join(mean_nombre_nom_districte, on='Nom_Districte', how='left')
DF = DF.withColumn('Nombre', F.when(DF['Nombre'].isNull(), DF['mean_nombre_nom_districte']).otherwise(DF['Nombre']))

# Verificar NAs després de la imputació per districte
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs en districte')

# Imprimir totes les files amb NA a la columna "Nombre"
DF.filter(DF.Nombre.isNull()).show()


# escalar obteninint mínim i màxims via query en funció de la unitat
min_max = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/compravenda_sup.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", 'SELECT "Superfície_mitjana_(m2_construïts)", min(Nombre) AS min_nombre , max(Nombre) AS max_nombre FROM RDD GROUP BY "Superfície_mitjana_(m2_construïts)"') \
  .load().collect()
  
# ver cuantos valores hay en min_max
print(f"Hi ha {len(min_max)} valors a min_max")

DF = DF.withColumn('Nombre', F.when(DF['Superfície_mitjana_(m2_construïts)'] == min_max[0][0], (DF['Nombre'] - min_max[0].min_nombre)/(min_max[0].max_nombre - min_max[0].min_nombre)).otherwise((DF['Nombre'] - min_max[1].min_nombre)/(min_max[1].max_nombre - min_max[1].min_nombre)))

# outliers
IQR = DF.approxQuantile('Nombre', [0.25, 0.75], 0.001)
IQR_values = IQR[1] - IQR[0]
        
DF = DF.withColumn('IQR', lit(IQR_values))
                
not_outliers = DF.filter(col('Nombre') >= IQR[0] - 1.5 * IQR_values)\
            .filter(col('Nombre') <= IQR[1] + 1.5 * IQR_values)

outliers = DF.subtract(not_outliers)
print(f"Hi ha {outliers.count()} valors sense outliers")
  
  

Hi ha 0 valors duplicats al DataFrame 'formatted_zone/compravenda_sup.db' de 876 valors totals
Hi ha 290 NAs
Hi ha 12 NAs en barri
Hi ha 12 NAs en districte
+-------------+---------+---+---------+--------------+----------+----------------------------------+------+---------------------+-------------------------+
|Nom_Districte|Nom_Barri|Any|Trimestre|Codi_Districte|Codi_Barri|Superfície_mitjana_(m2_construïts)|Nombre|mean_nombre_nom_barri|mean_nombre_nom_districte|
+-------------+---------+---+---------+--------------+----------+----------------------------------+------+---------------------+-------------------------+
+-------------+---------+---+---------+--------------+----------+----------------------------------+------+---------------------+-------------------------+

Hi ha 4 valors a min_max


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [10]:
spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/compravenda_sup.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", 'SELECT "Superfície_mitjana_(m2_construïts)", min(Nombre) AS min_nombre , max(Nombre) AS max_nombre FROM RDD WHERE Nombre <> "--" GROUP BY "Superfície_mitjana_(m2_construïts)"') \
  .load()\
  .show()

Py4JJavaError: An error occurred while calling o79.load.
: java.sql.SQLException: java.sql.SQLException: Binder Error: Referenced column "--" not found in FROM clause!
Candidate bindings: "RDD.Any"
LINE 1: ...truïts)", MIN(CAST(CASE WHEN Nombre != "--" THEN Nombre END AS FLOAT)) AS min_...
                                                  ^
	at org.duckdb.DuckDBPreparedStatement.prepare(DuckDBPreparedStatement.java:122)
	at org.duckdb.DuckDBPreparedStatement.<init>(DuckDBPreparedStatement.java:76)
	at org.duckdb.DuckDBConnection.prepareStatement(DuckDBConnection.java:76)
	at org.duckdb.DuckDBConnection.prepareStatement(DuckDBConnection.java:188)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:65)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:241)
	at org.apache.spark.sql.execution.datasources.jdbc.Jd

In [1]:
spark.stop()

NameError: name 'spark' is not defined